In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
%load_ext blackcellmagic

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import sys
import os
from functools import reduce
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
sns.set(style = 'darkgrid')

In [ ]:
sys.path.insert(0, os.getcwd())

In [ ]:
import extract
import preprocess as prep
import utilities as util
import model_compare as mod_com

bank_app - loanid, entered_date, bv decisions

predicon_model - loan_id, entered_date, count of scored applicants, #positive

loans - loanid, lead_time_added, #lender approved

## In a Nutshell

### Extracting Data

In [ ]:
bank_app = extract.extract_bankapp("'2020-06-09'")

In [ ]:
model = extract.extract_model()

In [ ]:
loan_history = extract.extract_loan_history("'2020-06-09'")

In [ ]:
loan_performance = extract.extract_performance("'2020-06-09'")

### Modifying Data

In [ ]:
modified_bank_app = prep.preprocess_bankapp_db(bank_app)

In [ ]:
modified_model_scores = prep.preprocess_model_db(model)

In [ ]:
modified_loan_history = prep.preprocess_loan_history_db(loan_history)

In [ ]:
modified_loan_performance = prep.preprocess_loan_performance(loan_performance)

### Analysing loanids which were not lender approved but our model approved

In [ ]:
dfs = [modified_bank_app, modified_model_scores, modified_loan_history,loan_performance]
merged_db = reduce(lambda left, right : pd.merge(left, right, on = 'LoanId', how = 'left'), dfs)

In [ ]:
merged_db['Decision'].fillna(-1, inplace = True)

In [ ]:
sub_cats = ['len_to_many_open_loans_101', 'len_return_found_102', 'len_missing_payments_103',
            'bk_verif_bal_on_payday_130', 'bk_verif_high_negative_curr_bal_131',
            'bk_verif_high_pct_negative_bal_132', 'bk_verif_bad_acc_type_133',
            'bk_verif_savings_acc_134', 'bk_verif_business_acc_135',
            'bk_verif_pre_debit_card_acc_136', 'bk_verif_stop_payment_or_revoked_137']

In [ ]:
merged_db = prep.preprocess_lender_reject_sub_categories(merged_db, sub_cats)

In [ ]:
lender_reject_model_approved = merged_db[(merged_db['Decision'] == 1) & (merged_db['LenderApproved'] == 0)]

In [ ]:
missing_decisions_loanids = merged_db[(merged_db['Decision'] == 1) & (merged_db['LenderApproved'] == 0) & 
                                      (merged_db['underwriting_final_decision'].isnull())]['LoanId'].values

In [ ]:
util.get_confusion_matrix(merged_db, norm = 'index')

In [ ]:
lender_reject_model_approved['underwriting_final_decision'].value_counts()

In [ ]:
util.get_no_sale_lender_bank_verification_subcategories(lender_reject_model_approved);

### Model performance metrics

In [ ]:
util.get_model_performance_wrt_matured_loans(merged_db, 'Positive');

In [ ]:
util.get_model_performance_wrt_matured_loans(merged_db, 'Neutral');